In [1]:
# to plot the rectangles
import matplotlib.pyplot as plt
# to parse the text file
import re
# to store the data as a dataframe
import pandas as pd
#  type hinting for the function
from typing import Optional, List, Tuple, Set
# default dictionary to store rectangles
from collections import defaultdict

import numpy as np
import graphviz as gv
import random 
import matplotlib.colors as mcolors
from enum import Enum


In [2]:
adjM = np.array([[0,2,1,0], [2,0,3,0], [1,3,0,2],[0,0,2,0]])
MAX_GAIN = np.max(np.sum(adjM, axis=1))
print(f"Max gain value: {MAX_GAIN}")
print("Creating an AdjList")
adjL = defaultdict(list)
for i in range(adjM.shape[0]):
    for j in range(i+1,adjM.shape[1]):
        if adjM[i][j] > 0:
            adjL[i].append((j,adjM[i][j]))
            adjL[j].append((i,adjM[i][j]))
print(f"Adj list: {adjL}")
nodes = list(adjL.keys())
print(f"Nodes:{nodes}")

Max gain value: 6
Creating an AdjList
Adj list: defaultdict(<class 'list'>, {0: [(1, 2), (2, 1)], 1: [(0, 2), (2, 3)], 2: [(0, 1), (1, 3), (3, 2)], 3: [(2, 2)]})
Nodes:[0, 1, 2, 3]


In [3]:
def random_partition(lst):
    # TODO REMOVE SEED
    random.seed(3)
    random.shuffle(lst.copy())
    midpoint = len(lst) // 2
    return set(lst[:midpoint]), set(lst[midpoint:])

def in_same_set(u,v,a,b):
    return u in a and v in a or u in b and v in b

In [4]:
a,b = random_partition(nodes)
print(f"Partitioning the nodes into two sets.\na->{a}\nb->{b}")

Partitioning the nodes into two sets.
a->{0, 1}
b->{2, 3}


In [5]:
gain = defaultdict(int)
cut_size = 0
bucket_a = [set() for _ in range(2*MAX_GAIN+1)]
bucket_b = [set() for _ in range(2*MAX_GAIN+1)]
for u in adjL:    
    for v,w in adjL[u]:
        if in_same_set(u,v,a,b):
            gain[u] -= w
        else:
            gain[u] += w
            cut_size += 1
    if u in a:
        bucket_a[gain[u]].add(u)
    else:
        bucket_b[gain[u]].add(u)
print(f"Initial gain: {gain}")
print(f"Initial cut size: {cut_size}")
print(f"Initial bucket_a: {bucket_a}")
print(f"Initial bucket_b: {bucket_b}")


Initial gain: defaultdict(<class 'int'>, {0: -1, 1: 1, 2: 2, 3: -2})
Initial cut size: 4
Initial bucket_a: [set(), {1}, set(), set(), set(), set(), set(), set(), set(), set(), set(), set(), {0}]
Initial bucket_b: [set(), set(), {2}, set(), set(), set(), set(), set(), set(), set(), set(), {3}, set()]


In [6]:
# TODO REMOVE THE AREA DICT 
area_dict = {0: 10, 1: 20, 2: 5, 3: 15}
area_dict

{0: 10, 1: 20, 2: 5, 3: 15}

In [7]:
size_a = sum([area_dict[u] for u in a])
size_b = sum([area_dict[u] for u in b])
print(f"Initial size_a: {size_a}")
print(f"Initial size_b: {size_b}")

Initial size_a: 30
Initial size_b: 20


In [8]:
def get_compliment_set(partition_set):
    return 'b' if partition_set == 'a' else 'a'

In [9]:
def find_maximum_gain():
    min_set = 'a' if size_a < size_b else 'b'
    size_min_set = eval(f'size_{min_set}')
    r = size_min_set/(size_a+size_b)
    print(f"ratio: {r}")
    if abs(r-0.5) <= 0.01:
        for g in range(MAX_GAIN,-MAX_GAIN-1,-1):
            if bucket_a[g]:
                return bucket_a[g], 'a'
            if bucket_b[g]:
                return bucket_b[g], 'b'
    else:
        compliment_set = get_compliment_set(min_set)
        bucket = eval(f"bucket_{compliment_set}")
        for g in range(MAX_GAIN,-MAX_GAIN-1,-1):
            if bucket[g]:
                return bucket[g],compliment_set


In [10]:
max_gain_set, max_gain_set_name = find_maximum_gain()
print(f"max_gain_set: {max_gain_set}")
print(f"max_gain_set_name: {max_gain_set_name}")
u = max_gain_set.pop()
print(f"u: {u}")


ratio: 0.4
max_gain_set: {1}
max_gain_set_name: a
u: 1


In [11]:
locked_cells = [(None,cut_size)]
locked_cells

[(None, 4)]

In [12]:
for v,w in adjL[u]:
    old_bucket = eval(f'bucket_{max_gain_set_name}')
    new_bucket = eval(f'bucket_{get_compliment_set(max_gain_set_name)}')
    if in_same_set(u,v,a,b):
        old_bucket[gain[v]].remove(v)
        gain[v] += 2*w
        old_bucket[gain[v]].add(v)
    else:
        new_bucket[gain[v]].remove(v)
        gain[v] -= 2*w
        new_bucket[gain[v]].add(v)

new_cut_set = cut_size - 2*gain[u]
gain.pop(u)
locked_cells.append((u,new_cut_set))
old_cell_set = eval(f'{max_gain_set_name}')
new_cell_set = eval(f'{get_compliment_set(max_gain_set_name)}')
old_cell_set.remove(u)
new_cell_set.add(u)
print(f"old_cell_set: {old_cell_set}")
print(f"new_cell_set: {new_cell_set}")
print(f"locked_cells: {locked_cells}")
print(f"gain: {gain}")
print(f"bucket_a: {bucket_a}")
print(f"bucket_b: {bucket_b}")


old_cell_set: {0}
new_cell_set: {1, 2, 3}
locked_cells: [(None, 4), (1, 2)]
gain: defaultdict(<class 'int'>, {0: 3, 2: -4, 3: -2})
bucket_a: [set(), set(), set(), {0}, set(), set(), set(), set(), set(), set(), set(), set(), set()]
bucket_b: [set(), set(), set(), set(), set(), set(), set(), set(), set(), {2}, set(), {3}, set()]
